#### 1. Импорты + insttall  Bertopic + загрузка  russian_stopwords


Корректная работа ноутбука предполагает установку папки модуля, являющейся пакетом, в каталог, содержащий этот ноутбук, а также наличие файла .env для хранения переменных окружения.
YC_API_KEY = '......'  для yndexgpt
FOLDER_ID  = '......'  id  фодера в yandexcloud

Yandex Foundation Models	Text generation	
0,20 ₽ 1k*unit	3 декабря 2024 г. (Стоимость генерации)

In [1]:
import warnings
import nltk
import torch
import  numpy as np
import re
import pandas as pd
import os
import tiktoken

from langchain_community.chat_models import ChatYandexGPT
from langchain_core.prompts import ChatPromptTemplate

from dotenv import load_dotenv,find_dotenv
from typing import Optional

from transformers import AutoTokenizer, pipeline
from pathlib import Path
from nltk.corpus import stopwords 

from  module import DictToDataFrameParser, InitBertopic

warnings.filterwarnings('ignore', category=UserWarning, module='transformers')
warnings.filterwarnings('ignore', category=FutureWarning, module='transformers')

C:\Users\dmitr\anaconda3\envs\env_NLP\Lib\site-packages\pydantic\_internal\_fields.py:132: UserWarning: Field "model_uri" in _BaseYandexGPT has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
C:\Users\dmitr\anaconda3\envs\env_NLP\Lib\site-packages\pydantic\_internal\_fields.py:132: UserWarning: Field "model_name" in _BaseYandexGPT has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
C:\Users\dmitr\anaconda3\envs\env_NLP\Lib\site-packages\pydantic\_internal\_fields.py:132: UserWarning: Field "model_version" in _BaseYandexGPT has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


DictDictToDataFrameParser:<br>
def __init__(self, path: Path, columns: Optional[dict[str, str]] = None):<br>
path - путь  *json

атрибуты:
- path (Path): Путь к JSON-файлу, из которого будет загружен словарь.
- columns (dict): Словарь, где ключи соответствуют ключам из JSON-структуры, 
                    а значения — названиям столбцов в первичном DataFrame.
- result_dict (dict): Десериализованный словарь, полученный из JSON-файла.
- df (DataFrame): DataFrame, созданный на основе данных из result_dict и columns
- name: Название чата

InitBertopic:

 def __init__(self,
             nr_topics: Optional[int] = None,
             top_n_words: int = 10,   
             model: Optional[str] = None, 
             umap_dict: Optional[dict] = None, 
             hdbscan_dict: Optional[dict] = None,
             vectorizer_dict: Optional[dict] = None, 
             tf_idf_dict: Optional[dict] = None) -> None:

Инициализирует Bertopic c заданными парамтрами,
атрибут:
- topic_mode - инициализированная модель Bertopic
- bertopic_dict - словарь спарметрами инициализированной модели

методы работы  berttopic:
**name_model** = InitBertopic(...)

- **name_model**.topic_model.fit_transform(text: list[str]) - кластеризация
- **name_model**.topic_model.get_topic_info() - результат
- **name_model**.topic_model.get_topic([int]) - ключевые слова топика
- **name_model**.topic_model.get_topic_freq() -датафрейм с меткой кластера и частотой

In [2]:

# stop words для 'Vectorizers in BERTopic'
try:
    russian_stopwords = stopwords.words("russian") 
except:
    nltk.download("stopwords")
    russian_stopwords = stopwords.words("russian")

In [3]:
# Проверка наличия .env  в  каталоге  проекта и установка переменных окружения True - успех!
load_dotenv(find_dotenv())

True

In [4]:
pattern  = r'[^\w,.\!? ]'
def cleaned_string(string: str, pattern: str = pattern) -> str: 
    '''
    По умолчанию  удаляются все символы, не являющиеся буквами, цифрами, пробелами и знаками 
                   препинания.
    Замена гиперссылок на  подстроку "гиперссылка"
    '''
    # Вопрос к ревью - замена  гипрссылок на ключевое слово гипрессылка стоит ли???
    string = re.sub(r'https?://\S+', 'гиперсссылка', string)
    return re.sub(pattern, '', string).strip()

In [4]:
#Для подготовки текста принята гипотеза (вопрос к ревью — как проверить?): текстовые признаки в наблюдениях являются зависимыми.
#Признаки, составляющие цепочку ответов (например, родительское сообщение и ответы на него), принадлежат одному кластеру.
#Для кластеризации выбираются независимые признаки, а также те, которые в результате среза не имеют ссылки на родительский признак,
#включая ответные сообщения на родительские сообщения, которые были удалены.


def text_preparation( df: pd.DataFrame, len_text: str = 3) -> tuple[list[int],list[str]]:
    ''' Подготавливает текстовые данные для передачи в BERTopic:
        применяет к каждому тексту функцию `cleaned_string`,
        и возвращает два списка:
        - список индексов для кластеризации,
        - очищенные строки (тексты), которые можно передать в модель.
        len_text -минимальная длина текста
    '''
    # Вопрос  ревью -каое значение len_text выбрать? (длина в символах не в словах!!!)
    if not isinstance(df, pd.DataFrame):
       raise TypeError("Expected DataFrame, got another type.")
    if  'text'not in df.columns:
       raise KeyError("Column 'text' is missing in the DataFrame")
    # Если в JSON есть reply_to_message_id, столбец будет типа int. (явно был приведен в DictDictToDataFrameParser)   
    if  'reply_to_message_id' in  df.columns and df_1.df.reply_to_message_id.dtype =='int64':  
        # список  значений df.reply_to_message_id  для которых нет исходного сообщения
        set_replay = set(df_1.df.reply_to_message_id)  - set(df_1.df.message_id) - {-1}
        df_without_replay = df_1.df.query('reply_to_message_id == -1 or reply_to_message_id in @set_replay')
        
        text = list(df_without_replay[df_without_replay['text'].str.len() >= len_text]['text'].items())
    else:
        text = list(df_1.df[df_1.df['text'].str.len() >= len_text]['text'].items())
           
    
    return  map(list,zip(*text)) 
    

### 2. Загрузка данных 

In [5]:
folder_ignore = '.ipynb_checkpoints'
path_list =[  i for i  in Path.cwd().rglob('*.json') if folder_ignore not in list(i.parent.parts) ]

# Список путей к файлам с json (парсим  рабочий каталог+ все  внего вложенные, кроме  '.ipynb_checkpoints')
[ (path_list, f'id_path:{i}')   for i, path_list      in enumerate(path_list)]

[(WindowsPath('C:/Users/dmitr/project_loc/masterskay/Masterskaya/data/example/result.json'),
  'id_path:0'),
 (WindowsPath('C:/Users/dmitr/project_loc/masterskay/Masterskaya/data/дата/result.json'),
  'id_path:1'),
 (WindowsPath('C:/Users/dmitr/project_loc/masterskay/Masterskaya/data/маркетинг/result.json'),
  'id_path:2'),
 (WindowsPath('C:/Users/dmitr/project_loc/masterskay/Masterskaya/data/менеджмент/result.json'),
  'id_path:3')]

**Число  в кортеже ( Path, int)  потребуется для выбора нужного файла .json**

In [6]:
INDEX =len(path_list)
while INDEX >= len(path_list) or INDEX <0:
    INDEX = int(input(f'Введи id_path (число от 0 до {len(path_list)-1}) и  Enter нажми'))
    if INDEX >= len(path_list):
        print( 'Введенное значение превышает допустимый диапазон')

Введи id_path (число от 0 до 3) и  Enter нажми 2


In [7]:
df_1 = DictToDataFrameParser(path_list[INDEX])
df_1.name

'💬 Marketing Practicum Chat'

In [259]:
df_1.df.head(5)

,date,chat_name,chat_id,sender_id,sender_type,message_id,type,reply_to_message_id,user_name,first_name,last_name,text
0,2025-02-01 07:35:54,💬 Marketing Practicum Chat,1782474569,1272493086,user,42471,message,-1,Pasha Sannikov,Pasha,Sannikov,"Доброе утро, люди! Это ОК, в РСЯ алгоритмы выб..."
1,2025-02-01 07:40:03,💬 Marketing Practicum Chat,1782474569,1272493086,user,42472,message,-1,Pasha Sannikov,Pasha,Sannikov,"А вот регионы, в которых живут ITшники с интер..."
2,2025-02-01 07:48:53,💬 Marketing Practicum Chat,1782474569,1272493086,user,42473,message,-1,Pasha Sannikov,Pasha,Sannikov,"Если смотреть по интересам, то кажется, что ау..."
3,2025-02-01 07:50:31,💬 Marketing Practicum Chat,1782474569,1272493086,user,42474,message,-1,Pasha Sannikov,Pasha,Sannikov,А путь пользователя почему-то выглядит так. Ко...
4,2025-02-01 08:12:42,💬 Marketing Practicum Chat,1782474569,1164775873,user,42475,message,-1,Данилов | Маркетолог,Данилов,|,Опять он


In [260]:
df_1.df.shape

(3680, 12)

In [14]:
LEN_TEXT = 3 
index_df, text = text_preparation(df_1.df, len_text= LEN_TEXT)
assert len(text) == len(index_df), (f'Lengths do not match: {len(text)} != {len(index_df)}')

index_df - список индексов для разметки  df (некоторые тексты не вошли  в итоговый корпус)

### 3. Bertopic, инициализация,  пробный topic modeling.

submodels of Bertopic:
1. Embedding Models
2. Dimensionality Reduction(UMAP)
3. Clustering (HDBSCAN)
4. Vectorizers(CountVectorizer)
5. c-TF-IDF (TF-IDF was adjusted to work on a cluster/categorical/topic level instead of a document level)  
6. Fine-tune Topics

In [16]:
# модель для Embedding Models
# model = 'sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2'
model = "DeepPavlov/rubert-base-cased-conversational"
#параметры модели понижения размерности
umap_dict = dict(n_neighbors=3, n_components=5, min_dist=0.0, metric='cosine')
#параметры модели кластеризации
hdbscan_dict = dict(min_cluster_size=10, metric='euclidean', cluster_selection_method='eom', prediction_data=True)
#параметры для Vectorizers
vectorizer_dict = dict(stop_words=russian_stopwords, ngram_range=(1, 3), min_df=1)
#параметры для TF_IDF
tf_idf_dict = dict(seed_words=["ссылка", "яндекс", "практикум", "стажировка", "курсы",'вакансия'],
                   seed_multiplier=2
                  )
                  
# nr_topics - количество  тем(ориентировочно)
# top_n_words - количество наиболее значимых слов для каждой темы.



In [17]:
NR_TOPIC = 10
TOP_N_WORDS =15
bertopic_model = InitBertopic(nr_topics=NR_TOPIC,
               top_n_words=TOP_N_WORDS,
               model=model,
               umap_dict=umap_dict, 
               hdbscan_dict=hdbscan_dict, 
               vectorizer_dict=vectorizer_dict, 
               tf_idf_dict=tf_idf_dict)

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased-conversational were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [18]:
bertopic_model.bertopic_dict

{'embedding_model': <module.bertopic_setup.CustomEmbedder at 0x1f64c400d10>,
 'nr_topics': 10,
 'top_n_words': 15,
 'umap_model': UMAP(metric='cosine', min_dist=0.0, n_components=5, n_neighbors=3),
 'hdbscan_model': HDBSCAN(min_cluster_size=10, prediction_data=True),
 'vectorizer_model': CountVectorizer(ngram_range=(1, 3),
                 stop_words=['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с',
                             'со', 'как', 'а', 'то', 'все', 'она', 'так', 'его',
                             'но', 'да', 'ты', 'к', 'у', 'же', 'вы', 'за', 'бы',
                             'по', 'только', 'ее', 'мне', ...]),
 'ctfidf_model': ClassTfidfTransformer(seed_words=['ссылка', 'яндекс', 'практикум', 'стажировка',
                                   'курсы', 'вакансия'])}

In [19]:
%%time
topics, probs = bertopic_model.topic_model.fit_transform(text)

CPU times: total: 5min 22s
Wall time: 3min 2s


In [280]:
bertopic_model.topic_model.get_topic_freq()

,Topic,Count
0,0,443
1,-1,325
2,1,276
4,2,109
3,3,48
5,4,33
6,5,23
8,6,13
7,7,11
9,8,11


In [20]:
# добавляем  в дтатфрейм( изменяем атрибут .df)
#столбцы topic - номер  кластера, probs - вероятность принадлежности к кластеру (важность?)
# Для зависмых текстов probs = 0 ( заглушка для дальнейшего отбора токенов d LLM, если  вокно не укладываемся)

if 'topic' in df_1.df.columns:
    df_1.df = df_1.df.drop(columns=['topic', 'probs'], errors='ignore')
df_1.df = df_1.df.join(pd.DataFrame({'topic': topics,'probs':np.round(probs,3)}, index=index_df))
df_1.df['probs'] = df_1.df['probs'].fillna(0)
df_1.df.sample(5)   
    

,date,chat_name,chat_id,sender_id,sender_type,message_id,type,reply_to_message_id,user_name,first_name,last_name,text,topic,probs
2493,2025-02-17 16:52:46,💬 Marketing Practicum Chat,1782474569,87502689,user,45100,message,-1,Андрей Дамаскин,Андрей,Дамаскин,В общем на сайте 3 счетчика Метрики!,0.0,1.0
2577,2025-02-18 17:08:59,💬 Marketing Practicum Chat,1782474569,5392944830,user,45192,message,45191,Олег,Олег,,"Но я не такой дурак, нашёл блокнот 😁",NaN,0.0
2468,2025-02-17 16:36:46,💬 Marketing Practicum Chat,1782474569,5392944830,user,45075,message,45072,Олег,Олег,,Они так и думают 😁 зачастую,NaN,0.0
2052,2025-02-13 11:33:45,💬 Marketing Practicum Chat,1782474569,87502689,user,44627,message,-1,Андрей Дамаскин,Андрей,Дамаскин,Потому что без качественных изменений на сайте...,1.0,1.0
727,2025-02-05 14:29:04,💬 Marketing Practicum Chat,1782474569,5392944830,user,43236,message,43235,Олег,Олег,,ЦРМ передаст офлайн конверсию в метрику,NaN,0.0


Алгоритм разметки зависимых сообщений с использованием полученных кластеров сталкивается с трудностью: некоторые сообщения ссылаются не на родительское сообщение, а на сообщение в цепочке ответов, которое, в свою очередь, также ещё не размечено

Наблюдается  что то вроде рекурсии  сначала размечаются ссобщения '1 уровня'  затем цикл повторяется и размечаютя сообщения  'второго уровня'  и так далее  критерий   ['topic'].sna()sum()перестает меняться.
Создаем словарь, где ключами являются номера топиков, а значениями — списки ID-сообщений, входящих в соответствующие кластеры. Если для какого-либо сообщения существует поле reply_message, и его ID присутствует в списке значений по соответствующему ключу, то этому сообщению присваивается значение этого ключа.
Проверяется условие: если количество  пропусков не изменилось значит процесс разметки окончен.

(Код тяжелый,  хотел бы более легкого ршения)

In [21]:
df_1.df.topic.isna().sum()

2388

In [22]:

while True:
    term = df_1.df['topic'].isna().sum()
    dict_1 = { i: df_1.df[df_1.df['topic'] == i]['message_id'].to_list() for i in  df_1.df.topic.dropna().unique()}
    for i in dict_1:
        df_1.df.loc[df_1.df['reply_to_message_id'].isin(dict_1[i]),'topic'] = i
    
    # Если количество NaN значений не изменилось, выходим из цикла
    if  term == df_1.df['topic'].isna().sum():
        break
    

In [23]:
# короткие  строки
df_1.df.topic.isna().sum()

163

### 4. LLM выделение темы из корпуса текстов

In [26]:
system_message = '''Выдели основную тему из приведённого текста.
Начни с краткого и ёмкого названия темы (от 5 до 10 слов), точно отражающего суть текста.
Затем выдели ключевые слова и фразы (5–10).
В конце сформулируй главную мысль.
Не добавляй лишнего текста.'''

user_message = "{text} \nФормат ответа: {{'Основная тема': '...', 'ключевые слова': [...], 'Главная мысль': '...'}}"

Выделение темы реализовано  с использованием  класса TopicLLMLL:

Выбрана модель ChatYandexGPT ( поддерживаются сообщения формата LangChain )
def __init__(self, task: str, format_user_messages: str):
task - ситемное  сообщение, format_user_messages - сообщение пользователя.
Реализован  магический метод __call__  парметр датафрейм  с обязательными полями 'topic',  'text'
В дальнейшем планируется вынести в TopicLLMll  в отдельный модуль

In [33]:
class TopicLLMll():
    def __init__(self, task: str, format_user_messages: str, max_tokens: int = 8000):
        self.max_tokens = max_tokens
        self.task = task
        self.format = format_user_messages
    system_message ='{task}'
    user_message = '{text}\n {format_user_messages}'
    user_message = user_message = "{text} "
    
    prompt_template = ChatPromptTemplate.from_messages([
        ('system', system_message),('user',user_message)])
   
    
    enc = tiktoken.get_encoding("cl100k_base")
    
    if os.getenv('FOLDER_ID') and os.getenv('YC_API_KEY'):
        chat_model = ChatYandexGPT(folder_id = os.getenv('FOLDER_ID'))
    else: 
        raise  ValueError("Both environment variables must be set: FOLDER_ID and YC_API_KEY")

    def __call__(self, df_top: pd.DataFrame):
        df_top =df_top.dropna()
        df_top = df_top.sort_values(by='probs', ascending=False)
        topic=[]
        for i in df_top['topic'].unique():
            
            text = ' '.join(df_top[df_top['topic']== i ]['text'].to_list())
            text = self.count_cut_text(text, self.max_tokens)
            
            prompt = self.prompt_template.invoke({'task': self.task, 'text':text})
            topic.append((self.chat_model.invoke(prompt).content, i))
        return topic
                         
    
    def count_cut_text(self, text, max_tokens ):
        tokens = self.enc.encode(text)[:max_tokens]
        return self.enc.decode(tokens)
    

In [34]:
llm_model =TopicLLMll(system_message, user_message)

In [35]:
topic_list = llm_model(df_1.df)

In [39]:
df_topic_list = pd.DataFrame(topic_list, columns=['name_topic', 'num_topic']) \
    .set_index('num_topic') \
    .join(df_1.df['topic'].value_counts().rename('Topic_Frequency')) \
    .reset_index()

In [43]:
with pd.option_context ('display.max_colwidth', None):
    display( df_topic_list.sort_values(by = 'Topic_Frequency', ascending=False))

,num_topic,name_topic,Topic_Frequency
1,0.0,"**Название темы:** «Маркетинг и реклама».\n\n**Ключевые слова и фразы:**\n* реклама;\n* маркетинг;\n* конверсия;\n* трафик;\n* боты;\n* анализ данных;\n* таргетирование.\n\n**Главная мысль:** обсуждение различных аспектов маркетинга и рекламы, включая стратегии привлечения трафика, использование ботов, анализ данных и таргетирование аудитории.",1623
0,1.0,"**Тема текста:** обсуждение различных аспектов интернет-маркетинга, рекламы и продвижения в сети.\n\n**Ключевые слова и фразы:**\n* интернет-маркетинг;\n* реклама;\n* Яндекс Директ;\n* Google Analytics;\n* DeepSeek;\n* ChatGPT;\n* продвижение в сети;\n* лиды;\n* CTR;\n* конверсия.\n\n**Главная мысль:** текст представляет собой набор сообщений и обсуждений на тему интернет-маркетинга и рекламы, в которых участники делятся опытом, советами и мнениями по улучшению эффективности своих кампаний и привлечению клиентов.",760
9,-1.0,"**Название темы:** «Маркетинг и реклама в современном бизнесе».\n\n**Ключевые слова и фразы:**\n* маркетинг;\n* реклама;\n* Яндекс Директ;\n* бизнес;\n* продажи;\n* продвижение.\n\n**Главная мысль:** в современном бизнесе большое внимание уделяется маркетингу и рекламе как основным инструментам продвижения товаров и услуг. Обсуждаются различные аспекты рекламы, включая использование Яндекс Директ и других платформ для привлечения клиентов.",539
6,2.0,"**Тема:** обсуждение проблем и вопросов, связанных с настройкой и использованием рекламных инструментов в интернете.\n\n**Ключевые слова и фразы:**\n* Яндекс Директ;\n* Метрика;\n* вебвизор;\n* ретаргетинг;\n* Google Ads;\n* Яндекс Аудитории;\n* CRM;\n* идентификация пользователей;\n* настройка рекламы;\n* учёт конверсий.\n\n**Главная мысль:** участники обсуждения делятся опытом и знаниями по настройке и использованию рекламных инструментов в интернете, обсуждают проблемы и находят решения.",232
5,4.0,**Основная тема текста:** привлечение новых участников в исследование и программу карьерного развития Практикума.\n\n**Ключевые слова и фразы:**\n* исследование;\n* анкетирование;\n* карьерное развитие;\n* программа;\n* трудоустройство;\n* фриланс.\n\n**Главная мысль:** команда Практикума проводит исследование и приглашает выпускников курсов присоединиться к программе карьерного развития для поиска работы или выхода на фриланс.,124
3,5.0,"**Тема:** поиск специалистов и обсуждение маркетинговых стратегий.\n\n**Ключевые слова и фразы:** директолог, SEO-специалист, маркетолог, таргет, лидогенерация, CRM, реклама в Telegram, хакатон, медиаплан.\n\n**Главная мысль:** в обсуждении участвуют специалисты и маркетологи, которые ищут коллег для совместной работы над проектами, делятся опытом и обсуждают актуальные вопросы в сфере маркетинга.",86
2,3.0,"В интернете есть много сайтов с информацией на эту тему. [Посмотрите, что нашлось в поиске](https://ya.ru)",76
8,6.0,"**Тема:** работа с веб-инструментами и программирование.\n\n**Ключевые слова и фразы:**\n* WordPress;\n* Метрика;\n* JavaScript;\n* Python;\n* программирование;\n* веб-инструменты.\n\n**Главная мысль:** текст содержит информацию о работе с различными веб-инструментами и программировании, включая установку счётчиков Метрики на сайт и примеры кода на JavaScript.",36
4,7.0,"**Тема:** обсуждение идентификаторов в сервисах Яндекса.\n\n**Ключевые слова и фразы:**\n* код;\n* yclid;\n* uid;\n* копилот;\n* php;\n* уникальные номера;\n* Яндекс.\n\n**Главная мысль:** в тексте обсуждаются идентификаторы, используемые в сервисах Яндекса, и их роль в программировании и отслеживании действий пользователей.",27
7,8.0,**Тема:** завершение общения и пожелание спокойной ночи.\n\n**Ключевые слова и фразы:**\n* завершение общения;\n* пожелание спокойной ночи;\n* идти на боковую.\n\n**Главная мысль:** автор прощается с участниками общения и желает им спокойной ночи.,14


Кластер -1.0 BERTopic оценивает как мусорный — без выраженной тематической структуры. 

Предметом  исследовния и настройки могут являться:
- первичная обработка текста(  оценка  важности сообщений,  отбор сообщений для кластеризации,  создание синтетических сообщений
                             для центровки кластеров);
- выбор модели для получения эмбеддингов;
- Настройка редьюкции размерности;
- Кластеризация: выбор алгоритма (HDBSCAN, KMeans и др.) и его параметров;
- выбор LLM : сравнение локальных (бюджетных) и облачных (API) решений с точки зрения качества, скорости и затрат;
- Prompt Engineering 


Отзывы, замечания предложения ревьера:












